# Module 3 Homework

In this homework, we're going to work with categorical variables, first ML models (Decision Trees), and hyperparameter tuning.

Please use the [Colab Module 3](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/03-modeling/Module_3_Colab_Time_Series_Modeling.ipynb) for all tasks to ensure you have the same dataframe used for the Modeling part, as covered during the lecture. 
We suggest copying and extending it (around "TODO" comments).

**HINT**: If you want to avoid data truncation in GitHub's UI, try either of the following options:
* Open the notebook in [Colab, using the GitHub link to the notebook](https://colab.research.google.com/github/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/03-modeling/Module_3_Colab_Time_Series_Modeling.ipynb).
* Clone the repository to a local folder and open the notebook in Jupyter Notebook.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier

## Question 1 (1 point): Dummies on Month and Week-of-Month

**Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable `is_positive_growth_5d_future`?**

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'.
Once you've generated the new set of variables, find the most correlated (in absolute value) one with `is_positive_growth_5d_future` and round it to 3 digits after the comma.

Suggested path to a solution:
- [[Source](https://stackoverflow.com/questions/25249033/week-of-a-month-pandas)] Use this formula to get the week of month for the datetime variable d: `(d.day-1)//7+1` 
- Define a new string variable for all month-week_of_month combinations. Append it to the CATEGORICAL features set. You should have 5 variables treated as CATEGORICAL now: 'Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom'. In the end, you should get 115 dummy features, including 60 (=12*5) week_month_of_week dummies.
- Use [pandas.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to generate dummies.
- Use [pandas.DataFrame.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) function (also used in [Code Snippet 1]) to get correlations with `is_positive_growth_5d_future`, filter out only variables representing the new dummy set, and sort it by absolute values (you can define a new column "abs_corr" in the dataframe with correlations), and find the highest value (among the new dummies features set).

**NOTE**: new dummies will be used as features in the next tasks, please leave them in the dataset.

In [2]:
df = pd.read_parquet("stocks_df_combined_2024_05_07.parquet.brotli")
df

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.088542,0.101563,0.088542,0.097222,0.060163,1.031789e+09,MSFT,1986,1986-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.097222,0.102431,0.097222,0.100694,0.062311,3.081600e+08,MSFT,1986,1986-03-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.100694,0.103299,0.100694,0.102431,0.063386,1.331712e+08,MSFT,1986,1986-03-01,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.102431,0.103299,0.098958,0.099826,0.061774,6.776640e+07,MSFT,1986,1986-03-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.099826,0.100694,0.097222,0.098090,0.060700,4.789440e+07,MSFT,1986,1986-03-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1.571996e+06,LT.NS,2024,2024-04-01,1,...,1.006530,1.011164,1.108923,0.936075,0.949809,0.956129,0.913106,0.850046,1.423982,2.158543
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3.748847e+06,LT.NS,2024,2024-05-01,3,...,0.946279,0.973473,1.053911,0.931945,1.014925,0.926103,0.916902,0.903379,1.369046,2.038296
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4.079696e+06,LT.NS,2024,2024-05-01,4,...,0.942513,0.967125,1.049197,0.946816,1.063704,1.037155,0.986425,0.953153,1.462818,2.180063
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2.614667e+06,LT.NS,2024,2024-05-01,0,...,0.936187,0.975418,1.027877,0.952887,0.986426,1.004327,0.989362,0.916771,1.465996,2.219715


In [3]:
GROWTH = [g for g in df.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']
TO_PREDICT = [g for g in df.keys() if (g.find('future')>=0)]
df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']
TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
                        'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
                        'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
                        'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
                        'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
                        'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
                        'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
                        'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
                        'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']
TECHNICAL_PATTERNS = [g for g in df.keys() if g.find('cdl')>=0]
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
         'DGS1', 'DGS5', 'DGS10']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO


def create_week_of_month(row):
    month = row.month_name()
    week = (row.day - 1) // 7 + 1
    return month + "_w" + str(week)
df["month_wom"] = df["Date"].apply(create_week_of_month)

CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom']
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
OTHER = [k for k in df.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]

df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')
DUMMIES = dummy_variables.keys().to_list()
df_with_dummies = pd.concat([df, dummy_variables], axis=1)
df_with_dummies = df_with_dummies[df["Date"]>='2000-01-01']

/tmp/ipykernel_26744/3093481184.py:4: RuntimeWarning: divide by zero encountered in log
  df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))
/tmp/ipykernel_26744/3093481184.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['March' 'March' 'March' ... 'May' 'May' 'May']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[:,'Month'] = df.Month.dt.strftime('%B')
/tmp/ipykernel_26744/3093481184.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['3' '4' '0' ... '4' '0' '1']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)


In [4]:
corr_df = df_with_dummies[DUMMIES + ["is_positive_growth_5d_future"]].corr()
growth_5d_corr = corr_df["is_positive_growth_5d_future"].drop('is_positive_growth_5d_future')
max_idx = growth_5d_corr.abs().idxmax()
print(f"The CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable ('is_positive_growth_5d_future') is {growth_5d_corr[max_idx]:.3f}")

The CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable ('is_positive_growth_5d_future') is -0.035


## Question 2 (2 points): Define new "hand" rules on macro and technical indicators variables

**What is the precision score for the best of the NEW predictions (pred3 or pred4), rounded to 3 digits after the comma?**

Let's utilize the knowledge from the visualised tree (clf10) (Code Snippet 5: 1.4.4 Visualisation):

* You're asked to define two new 'hand' rules (leading to 'positive' subtrees): 
  - `pred3_manual_gdp_fastd`: (gdppot_us_yoy <= 0.027) & (fastd >= 0.251)
  - `pred4_manual_gdp_wti_oil`: (gdppot_us_yoy >= 0.027) & (growth_wti_oil_30d <= 1.005)

* Extend the Code Snippet 3 (Manual "hand rule" predictions): Calculate and add new rules (pred3 and pred4) to the dataframe.You should notice that one of the predictions doesn't have any positive predictions on TEST dataset (while it has many on TRAIN+VALIDATION). 

* Debug: check in the `new_df` and the original dataset/data generation process that we didn't make any mistakes during the data transformation step.

* Explain why this can happen even if there are no errors in the data features.

In [5]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)
new_df = df_with_dummies.copy()
new_df

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,month_wom_October_w2,month_wom_October_w3,month_wom_October_w4,month_wom_October_w5,month_wom_September_w1,month_wom_September_w2,month_wom_September_w3,month_wom_September_w4,month_wom_September_w5,split
3490,58.687500,59.312500,56.000000,58.281250,36.065567,53228400.0,MSFT,2000,January,0,...,0,0,0,0,0,0,0,0,0,train
3491,56.781250,58.562500,56.125000,56.312500,34.847271,54119000.0,MSFT,2000,January,1,...,0,0,0,0,0,0,0,0,0,train
3492,55.562500,58.187500,54.687500,56.906250,35.214706,64059600.0,MSFT,2000,January,2,...,0,0,0,0,0,0,0,0,0,train
3493,56.093750,56.937500,54.187500,55.000000,34.035072,54976600.0,MSFT,2000,January,3,...,0,0,0,0,0,0,0,0,0,train
3494,54.312500,56.125000,53.656250,55.718750,34.479843,62013600.0,MSFT,2000,January,4,...,0,0,0,0,0,0,0,0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422,3639.000000,3648.949951,3584.050049,3594.300049,3594.300049,1571996.0,LT.NS,2024,April,1,...,0,0,0,0,0,0,0,0,0,test
5423,3590.050049,3634.149902,3576.050049,3599.500000,3599.500000,3748847.0,LT.NS,2024,May,3,...,0,0,0,0,0,0,0,0,0,test
5424,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696.0,LT.NS,2024,May,4,...,0,0,0,0,0,0,0,0,0,test
5425,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667.0,LT.NS,2024,May,0,...,0,0,0,0,0,0,0,0,0,test


In [6]:
new_df['pred0_manual_cci'] = (new_df.cci>200).astype(int)
new_df['pred1_manual_prev_g1'] = (new_df.growth_1d>1).astype(int)
new_df['pred2_manual_prev_g1_and_snp'] = ((new_df['growth_1d'] > 1) & (new_df['growth_snp500_1d'] > 1)).astype(int)
new_df["pred3_manual_gdp_fastd"] = (new_df["gdppot_us_yoy"] <= 0.027) & (new_df["fastd"] >= 0.251).astype(int)
new_df["pred4_manual_gpd_wti_oil"] = (new_df["gdppot_us_yoy"] >= 0.027) & (new_df["growth_wti_oil_30d"] <= 1.005).astype(int)

In [7]:
test_idx = new_df.split.isin(['test'])
pred3_score = precision_score(new_df.loc[test_idx, "is_positive_growth_5d_future"], new_df.loc[test_idx, "pred3_manual_gdp_fastd"], zero_division=0)
pred4_score = precision_score(new_df.loc[test_idx, "is_positive_growth_5d_future"], new_df.loc[test_idx, "pred4_manual_gpd_wti_oil"], zero_division=0)
print(f"The precision score for the best of the NEW variables (pred3 or pred4) is {max(pred3_score, pred4_score):.3f}")

The precision score for the best of the NEW variables (pred3 or pred4) is 0.555


## Question 3 (1 point): Unique correct predictions from a 10-levels deep Decision Tree Classifier (pred5_clf_10) 

**What is the total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4)?**

NOTE: please include `random_state=42` to Decision Tree Classifier init function (line `clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)`) to ensure everyone gets the same results.

Suggested solution:
* Step1: Rewrite the '1.4.3 Inference for a decision tree' piece for the Decision Tree Classifier with max_depth=10 (clf_10), so that you fit the model on TRAIN+VALIDATION sets (unchanged from the lecture), but predict on the whole set X_all (to be able to define a new column 'pred5_clf_10' in the dataframe new_df). Here is the [link](https://stackoverflow.com/questions/40729162/merging-results-from-model-predict-with-original-pandas-dataframe) with explanation. It will solve the problem in 1.4.5 when predictions were made only for Test dataset and couldn't be easily joined with the full dataset. 

* Step2: Once you have it, define a new column 'only_pred5_is_correct' similar to 'hand' prediction rules with several conditions: is_positive_growth_5d_future AND is_correct_pred5 should be equal 1, while all other predictions is_correct_pred0..is_correct_pred4 should be equal to 0.

* Step3: Convert 'only_pred5_is_correct' column from bool to int, and find how many times it is equal to 1 in the TEST set. Write down this as an answer.

ADVANCED: define a function that can be applied to the whole row of predictions ([a few examples of pandas-apply-row-functions](https://sparkbyexamples.com/pandas/pandas-apply-function-to-every-row/)) and can find whether some prediction 'predX' (where X is one of the predictions) is uniquely correct. It should work even if there are 100 predictions available, so that you don't define manually the condition for 'predX'.  

In [8]:
PREDICTIONS = [k for k in new_df.keys() if k.startswith('pred')]
PREDICTIONS

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_gdp_fastd',
 'pred4_manual_gpd_wti_oil']

In [9]:
features_list = NUMERICAL + DUMMIES
to_predict = 'is_positive_growth_5d_future'
new_df.replace([np.inf, -np.inf], np.nan, inplace=True)
new_df.fillna(0, inplace=True)

train_df = new_df[new_df.split.isin(['train','validation'])]
test_df = new_df[new_df.split.isin(['test'])]
X_train = train_df[features_list]
X_test = test_df[features_list]
y_train = train_df[to_predict]
y_test = test_df[to_predict]
SEED = 42

In [10]:
clf_10 = DecisionTreeClassifier(max_depth=10, random_state=SEED)
clf_10.fit(X_train, y_train)

test_idx = new_df["split"].isin(['test'])
new_df.loc[test_idx, "pred5_clf_10"] = clf_10.predict(X_test)

def check_only_predx_is_correct(row, check_pred, other_preds, to_predict):
    correct_pred = (row[check_pred] == row[to_predict])
    other_preds_incorrect = all(row[pred] != row[to_predict] for pred in other_preds)
    return correct_pred and other_preds_incorrect

x = 5
other_preds = []
for col in new_df.columns:
    if col.startswith(f"pred{x}"):
        check_pred = col
    elif col.startswith("pred"):
        other_preds.append(col)
new_col = f'only_pred_{x}_is_Correct'
new_df.loc[test_idx, new_col] = new_df[test_idx].apply(check_only_predx_is_correct, axis=1, check_pred=check_pred, other_preds=other_preds, to_predict=to_predict)

print(f"The total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4) is {new_df[new_col].sum()}")

The total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4) is 1


## Question 4: (2 points) Hyperparameter tuning for a Decision Tree

**What is the optimal tree depth (from 1 to 20) for a DecisionTreeClassifier?**

NOTE: please include `random_state=42` to Decision Tree Classifier init function (line `clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)`) to ensure consistency in results.

Follow these steps to find the optimal `max_depth`:
* Iterate through `max_depth` values from 1 to 20.
* Train the Decision Tree Classifier with the current `max_depth` parameter.
* Optionally, visualize how the 'head' of each fitted tree changes with more advanced (=deep) trees. You can use the [`sklearn.tree.plot_tree()`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) function, or the compact way  with the `export_text()` functionality ([Stack Overflow example](https://stackoverflow.com/questions/20156951/how-do-i-find-which-attributes-my-tree-splits-on-when-using-scikit-learn)):
  ```
  from sklearn.tree import export_text
  tree_rules = export_text(model, feature_names=list(X_train), max_depth=3)
  print(tree_rules)
  ```
* Calculate the precision score (you can use the function [sklearn.metrics.precision_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html)) on the TEST dataset for each of the fitted trees. You can also compare it with the precision score on a VALIDATION dataset, which is included to the training phase (to have more data to train on). You should see that the precision score on a VALIDATION set starts to grow with the complexity of a tree (overfit), which isn't seen on the precision score on TEST.
* Identify the optimal `max_depth`, where the  precision score is the highest on the TEST dataset. Record this value as  **best_max_depth** and submit as an answer.
* Make predictions on all records (TRAIN+VALIDATION+TEST) and add the new prediction `pred6_clf_best` to the dataframe `new_df`.

Additionally, compare the precision score of the tuned decision tree with previous predictions. You should observe an improvement (>0.58, or more than 58% precision), indicating that the tuned tree outperforms previous manual "hand" rules and Decision Tree predictions.

ADVANCED: Read more about different aspects of [scikit-learn Decision Trees](https://scikit-learn.org/stable/modules/tree.html). Draw a line of precision/accuracy vs. max_depth and note whether there's a saturation point of precision/accuracy as max_depth increases. In theory, there should be a trade-off between better fitting (=more complex trees) and generalization.

In [11]:
max_depth = 20
test_precision = []

for depth in range(1, max_depth+1):
    clf = DecisionTreeClassifier(max_depth=depth, random_state=SEED)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precision = precision_score(y_test, y_pred)
    test_precision.append(precision)

print(f"The optimal tree depth (from 1 to 20) for a DecisionTreeClassifier is {np.argmax(test_precision) + 1}")

The optimal tree depth (from 1 to 20) for a DecisionTreeClassifier is 15


## [EXPLORATORY] Question 5: What data is missing? 

Now that you have some insights from the correlation analysis and the Decision Trees regarding the most influential variables, suggest new indicators you would like to include in the dataset and explain why.

You can also propose something entirely different based on your intuition, but it should be relevant to the shared dataset of the largest Indian, EU, and US stocks. If you choose this approach, please specify the data source as well.

## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw03